In [31]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

## Read the data and select the high dimension columns

In [1]:
import pandas as pd
df = pd.read_csv('D:/Emory/ML1/final_project/Train000.csv', header=None)
test = pd.read_csv('D:/Emory/ML1/final_project/Train004.csv', header=None, nrows=100000)
comparison_001 = pd.read_csv('D:/Emory/ML1/final_project/Train004.csv', header=None)

df.columns = ['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']
test.columns = ['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']
comparison_001.columns = ['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']


### your code here: read in the test data

df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,13908270110995968466,0,14102516,1005,0,85f751fd,c4e18dd6,50e219e0,92f5800b,ae637522,...,1,3,21189,320,50,2424,1,161,100193,71
1,10258522767243640572,0,14102819,1010,1,85f751fd,c4e18dd6,50e219e0,3fd3187a,7801e8d9,...,4,3,16252,320,50,1809,0,171,100162,126
2,7801328047195758573,0,14102516,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,-1,79
3,16046150418661423859,0,14102410,1005,1,92c7cbe7,6e882918,75fa27f6,ecad2386,7801e8d9,...,1,0,16615,320,50,1863,3,39,-1,23
4,12577991248190121985,0,14102316,1005,0,85f751fd,c4e18dd6,50e219e0,7358e05e,b9528b13,...,1,0,1039,320,50,178,3,1327,100148,15


## Transform the Data

### change the categories -- if the category has less than 100 rows, we change it to "others"

In [2]:


for i in df.columns[2:]:
    value_counts_i = dict(df[i].value_counts())
    dict_i_100 = {k: v for k, v in value_counts_i.items() if v > 100}
    df.loc[~df[i].isin(dict_i_100.keys()), i] = 'other'
    test.loc[~test[i].isin(dict_i_100.keys()),i] = 'other'
    comparison_001.loc[~comparison_001[i].isin(dict_i_100.keys()),i] = 'other'
    
    ### your code here: transform the test data (just follow the code above)
    
    
    

### drop the columns (device_ip, id, click, hour) in the predictors

In [3]:
df_high_dim_cate = df.loc[:, [i for i in df.columns if i not in ['device_ip','id','click','hour']]]
test_high_dim_cate = test.loc[:, [i for i in df.columns if i not in ['device_ip','id','click','hour']]]
comparison_high_dim_cate = comparison_001.loc[:, [i for i in df.columns if i not in ['device_ip','id','click','hour']]]



### your code here (drop the columns):




df_high_dim_cate.head()

,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1005,0,85f751fd,c4e18dd6,50e219e0,92f5800b,ae637522,0f2161f8,a99f214a,0bcabeaf,1,3,21189,320,50,2424,1,161,100193,71
1,1010,1,85f751fd,c4e18dd6,50e219e0,3fd3187a,7801e8d9,cef3e649,other,f07e20f8,4,3,other,320,50,1809,0,171,100162,126
2,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,edead9f4,1,0,15706,320,50,1722,0,35,-1,79
3,1005,1,92c7cbe7,6e882918,75fa27f6,ecad2386,7801e8d9,07d7df22,a99f214a,dc70b0f9,1,0,16615,320,50,1863,3,39,-1,23
4,1005,0,85f751fd,c4e18dd6,50e219e0,7358e05e,b9528b13,cef3e649,other,90df9d44,1,0,1039,320,50,178,3,1327,100148,15


### LabelEncoding -- follow the training data

In [4]:

from sklearn import metrics, preprocessing


features = df_high_dim_cate.columns
for feat in features:
    lbl_enc = preprocessing.LabelEncoder()
    lbl_enc.fit(df_high_dim_cate[feat].fillna("-1").astype(str).values)
    df_high_dim_cate[feat] = lbl_enc.transform(df_high_dim_cate[feat].fillna("-1").astype(str).values)
    test_high_dim_cate[feat] = lbl_enc.transform(test_high_dim_cate[feat].fillna("-1").astype(str).values)
    comparison_high_dim_cate[feat] = lbl_enc.transform(comparison_high_dim_cate[feat].fillna("-1").astype(str).values)
    
    ### your code here: labelencoding the data -- just use transform, do not use fit, since it's based on the training data
    
    
    

## Modelling Part, there is no need to change any of these, just read the h5 file I uploaded and make predictions

In [5]:
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils




import numpy as np
def create_model(data, catcols):    
    inputs = []
    outputs = []
    for c in catcols:
        num_unique_values = int(data[c].nunique())
        embed_dim = int(min(np.ceil((num_unique_values)/2), 30))
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(num_unique_values + 1, embed_dim, name=c)(inp)
        out = layers.SpatialDropout1D(0.3)(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)
    
    x = layers.Concatenate()(outputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(100, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=y)
    return model


In [19]:
from tensorflow import config
physical_devices = config.list_physical_devices('GPU') 

config.experimental.set_visible_devices(devices = physical_devices[0], device_type='GPU')


In [27]:
model = create_model(data = df_high_dim_cate, catcols = df_high_dim_cate.columns)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_101 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_102 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_103 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_104 (InputLayer)          [(None, 1)]          0                                            
____________________________________________________________________________________________

## transform the data type, follow the same code

In [57]:
X_train = [df_high_dim_cate.loc[:, features].values[:, k] for k in range(df_high_dim_cate.loc[:, features].values.shape[1])]
y_train = df.click

X_val = [test_high_dim_cate.loc[:, features].values[:, k] for k in range(test_high_dim_cate.loc[:, features].values.shape[1])]
y_val = test.click

X_comparison = [comparison_high_dim_cate.loc[:, features].values[:, k] for k in range(comparison_high_dim_cate.loc[:, features].values.shape[1])]
y_comparison = comparison_001.click


### your code here: do the same thing



In [13]:
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [50]:
with tf.device('/gpu:0'):
    model = create_model(data = df_high_dim_cate, catcols = df_high_dim_cate.columns)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    StopRule = EarlyStopping(monitor='val_loss',mode='min',patience=15,min_delta=0.0)
    FitHist = model.fit(X_train,y_train,validation_data=(X_val, y_val), \
                    epochs=1000,batch_size=512,verbose=1, \
                    callbacks=[StopRule])

Epoch 1/1000
1954/1954 [==============================] - 64s 32ms/step - loss: 0.4294 - accuracy: 0.8201 - val_loss: 0.4038 - val_accuracy: 0.8316
Epoch 2/1000
1954/1954 [==============================] - 61s 31ms/step - loss: 0.4039 - accuracy: 0.8323 - val_loss: 0.4018 - val_accuracy: 0.8320
Epoch 3/1000
1954/1954 [==============================] - 61s 31ms/step - loss: 0.4014 - accuracy: 0.8327 - val_loss: 0.4000 - val_accuracy: 0.8341
Epoch 4/1000
1954/1954 [==============================] - 61s 31ms/step - loss: 0.4001 - accuracy: 0.8330 - val_loss: 0.4000 - val_accuracy: 0.8342
Epoch 5/1000
1954/1954 [==============================] - 61s 31ms/step - loss: 0.3990 - accuracy: 0.8334 - val_loss: 0.4001 - val_accuracy: 0.8342
Epoch 6/1000
1954/1954 [==============================] - 61s 31ms/step - loss: 0.3981 - accuracy: 0.8333 - val_loss: 0.3989 - val_accuracy: 0.8342
Epoch 7/1000
1954/1954 [==============================] - 61s 31ms/step - loss: 0.3972 - accuracy: 0.8335 - val_

In [51]:
model.save("my_h5_model.h5")

In [58]:
comparison_pred = model.predict(X_comparison)

In [60]:
from sklearn.metrics import log_loss
log_loss(y_comparison, comparison_pred)

0.3973975775583769

## Read in the data and make predictions

In [45]:
from tensorflow import keras
reconstructed_model = keras.models.load_model("my_h5_model.h5")

In [66]:
### your code here: model.predict(...)

[<tf.Variable 'C1/embeddings:0' shape=(8, 4) dtype=float32, numpy=
 array([[ 0.910998  , -1.1575747 ,  1.2261786 , -0.8755283 ],
        [-0.01273742,  0.12044442, -0.10034866,  0.04664685],
        [ 0.04611974, -0.03842705,  0.01571357, -0.01134393],
        [ 0.73294896, -0.17232428,  0.2971256 , -0.7452355 ],
        [ 0.00948476,  0.19930771, -0.34358755,  0.3484934 ],
        [-0.14136449,  0.09384753, -0.04167957,  0.32457766],
        [ 0.6955031 , -0.618053  , -0.69611716, -0.64969164],
        [ 0.00708244,  0.04406519, -0.01632124,  0.04751069]],
       dtype=float32)>,
 <tf.Variable 'banner_pos/embeddings:0' shape=(8, 4) dtype=float32, numpy=
 array([[ 0.06890204,  0.05545846, -0.05514494,  0.08713716],
        [-0.5221489 , -0.41675794,  0.5722657 , -0.55480224],
        [ 0.03099139, -0.00706479, -0.39110678,  0.1170115 ],
        [-0.24975501, -0.14424518, -0.10202426, -0.28511366],
        [-0.5619948 , -0.50393647,  0.53831106, -0.36254826],
        [ 0.10868784, -0.52

## Combine the prediction and id together and save as a csv_file

In [ ]:
### your code here: 




## Include Weekday and Hours

In [6]:
train_time = pd.DataFrame({'t_hour':df.hour.apply(lambda x: str(x)[-2:]),\
              't_weekday':df.hour.apply(lambda x: pd.to_datetime(str(20)+str(x)[:-2], format='%Y%m%d').day_of_week)})

test_time = pd.DataFrame({'t_hour':test.hour.apply(lambda x: str(x)[-2:]),\
              't_weekday':test.hour.apply(lambda x: pd.to_datetime(str(20)+str(x)[:-2], format='%Y%m%d').day_of_week)})

comparison_time = pd.DataFrame({'t_hour':comparison_001.hour.apply(lambda x: str(x)[-2:]),\
              't_weekday':comparison_001.hour.apply(lambda x: pd.to_datetime(str(20)+str(x)[:-2], format='%Y%m%d').day_of_week)})

In [7]:
df_high_dim_cate_with_time = pd.concat([df_high_dim_cate, train_time], axis=1)
test_high_dim_cate_with_time = pd.concat([test_high_dim_cate, train_time], axis=1)
comparison_high_dim_cate_with_time = pd.concat([comparison_high_dim_cate, train_time], axis=1)

In [8]:
features_with_time = df_high_dim_cate_with_time.columns

In [16]:
X_train = [df_high_dim_cate_with_time.loc[:, features_with_time].values[:, k] for k in range(df_high_dim_cate_with_time.loc[:, features_with_time].values.shape[1])]
y_train = df.click

X_val = [test_high_dim_cate_with_time.loc[:, features_with_time].values[:, k] for k in range(test_high_dim_cate_with_time.loc[:, features_with_time].values.shape[1])]
y_val = test.click

#X_comparison = [comparison_high_dim_cate_with_time.loc[:, features_with_time].values[:, k] for k in range(comparison_high_dim_cate_with_time.loc[:, features_with_time].values.shape[1])]
#y_comparison = comparison_001.click

In [18]:
X_train = X_train.astype('float')
X_val = X_val.

[array([2, 5, 2, ..., 2, 2, 2], dtype=object),
 array([0, 1, 0, ..., 1, 0, 0], dtype=object),
 array([183, 183, 39, ..., 121, 183, 213], dtype=object),
 array([211, 211, 254, ..., 13, 211, 181], dtype=object),
 array([4, 4, 1, ..., 11, 4, 11], dtype=object),
 array([124, 59, 219, ..., 219, 50, 219], dtype=object),
 array([21, 13, 13, ..., 13, 27, 13], dtype=object),
 array([2, 10, 0, ..., 0, 2, 0], dtype=object),
 array([3, 6, 3, ..., 3, 3, 3], dtype=object),
 array([50, 807, 797, ..., 116, 177, 459], dtype=object),
 array([1, 2, 1, ..., 1, 1, 1], dtype=object),
 array([2, 2, 0, ..., 0, 0, 0], dtype=object),
 array([219, 678, 19, ..., 35, 22, 309], dtype=object),
 array([2, 2, 2, ..., 2, 2, 0], dtype=object),
 array([3, 3, 3, ..., 3, 3, 1], dtype=object),
 array([110, 25, 16, ..., 29, 18, 147], dtype=object),
 array([1, 0, 0, ..., 3, 3, 0], dtype=object),
 array([8, 12, 26, ..., 28, 12, 26], dtype=object),
 array([105, 86, 0, ..., 0, 66, 47], dtype=object),
 array([44, 9, 45, ..., 28, 

In [17]:
with tf.device('/gpu:0'):
    model_with_time = create_model(data = df_high_dim_cate_with_time, catcols = df_high_dim_cate_with_time.columns)
    model_with_time.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    StopRule = EarlyStopping(monitor='val_loss',mode='min',patience=15,min_delta=0.0)
    FitHist = model_with_time.fit(X_train,y_train,validation_data=(X_val, y_val), \
                    epochs=1000,batch_size=512,verbose=1, \
                    callbacks=[StopRule])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).